In [ ]:
# questions and personal notes
#problems with reading this as $600 value for W/D - linear regression
#('w/d hookups', -358.55112862600521), ('w/d in unit', 323.94738691104828),

#Still need to remove (dummmy variable values that have only a few instances - e.g. only one person from city)

In [1]:
# imports
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from math import exp

# allow plots to appear directly in the notebook
import seaborn as sns
%matplotlib inline

In [3]:
data = pd.read_csv(r'C:\Users\alsherman\Desktop\GitHub\DataScience_GeneralAssembly\Data\Cleaned_Data_May_18.csv')

In [4]:
data.describe()

,Unnamed: 0,ID,latitude,location_data_accuracy,longitude,average_image_size,bathroom,bedroom,cat,dog,...,price,smoking,square_footage,grocery_list,gym_list,movie_theatre_list,subway_station_list,Barnes_and_Nobles_list,Deloitte_list,Starbucks_list
count,1,1,1.000000,1,1.000000,1,1.0,1,1,1,...,1,1,0,1,1,1,1,1,1,1
mean,0,5005421838,39.118608,0,-77.211356,270000,2.5,3,0,0,...,1750,0,NaN,0,0,0,0,0,0,0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0,5005421838,39.118608,0,-77.211356,270000,2.5,3,0,0,...,1750,0,NaN,0,0,0,0,0,0,0
25%,0,5005421838,39.118608,0,-77.211356,270000,2.5,3,0,0,...,1750,0,NaN,0,0,0,0,0,0,0
50%,0,5005421838,39.118608,0,-77.211356,270000,2.5,3,0,0,...,1750,0,NaN,0,0,0,0,0,0,0
75%,0,5005421838,39.118608,0,-77.211356,270000,2.5,3,0,0,...,1750,0,NaN,0,0,0,0,0,0,0
max,0,5005421838,39.118608,0,-77.211356,270000,2.5,3,0,0,...,1750,0,NaN,0,0,0,0,0,0,0


In [4]:
# Transform categorical fields to numeric for scikit-learn 

data.country = np.where(data.country == 'US',1,0)
data.availability = np.where(data.availability == 'available now',1,0)

In [5]:
# Create dummy variables

data = pd.concat([data, pd.get_dummies(data.housing_type)], axis=1)
data = pd.concat([data, pd.get_dummies(data.laundry)], axis=1)
data = pd.concat([data, pd.get_dummies(data.parking)], axis=1)
data = pd.concat([data, pd.get_dummies(data.city)], axis=1)
data = pd.concat([data, pd.get_dummies(data.state)], axis=1)

data[0:1]

,Unnamed: 0,ID,city,country,latitude,location_data_accuracy,longitude,state,availability,average_image_size,...,md,park,potomac,rainier,ridge,spring,springs,va,village,washington
0,0,4959351766,NaN,0,0,0,0,NaN,1,270000,...,0,0,0,0,0,0,0,0,0,0


In [6]:
feature_cols = ['latitude',	 'location_data_accuracy',	 'longitude',	 'average_image_size',	 'bedroom',	 'cat',	 'dog',	 'image_number',	 'smoking',	 'square_footage',	 'grocery_list',	 'gym_list',	 'movie_theatre_list',	 'train_station_list',	 'airport_list',	 'subway_station_list',	 'Barnes_and_Nobles_list',	 'Deloitte_list',	 'Starbucks_list',	 'No_Housing_Data',	 'apartment',	 'condo',	 'cottage/cabin',	 'duplex',	 'flat',	 'house',	 'in-law',		
 'townhouse',	 'No_Laundry_Data',	 'laundry in bldg',	 'laundry on site',	 'w/d hookups',	 'w/d in unit',	 'No_Parking_Data',	 'attached garage',	 'carport',	 'detached garage',	 'off-street parking',	 'street parking',	 'aldie',	 'alexandria',	 'annandale',	 'arlington',	 'ashburn',	 'baltimore',	 'beltsville',	 'bethesda',	 'bladensburg',	 'bowie',	 'broadlands',	 'capitol',	 'centreville',	 'chantilly',	 'chevy',	 'clinton',	 'college',
 'columbia',	 'crofton',	 'dale',	 'damascus',	 'dumfries',	 'elkridge',	 'fairfax',	 'falls',	 'forestville',	 'frederick',	 'fredericksburg',	 'gaithersburg',	 'germantown',	 'greenbelt',	 'hamilton',	 'harpers',	 'herndon',	 'hyattsville',	 'kensington',	 'landover',	 'largo',	 'laurel',	 'laytonsville',	 'leesburg',	 'lorton',	 'manassas',	 'mclean',	 'montgomery',	 'n',
 'north',	 'northwest',	 'oak',	 'oakton',	 'oxon',	 'potomac',	 'reston',	 'riverdale',	 'rockville',	 'rosslyn',	 'silver',	 'springfield',	 'stafford',	 'sterling',	 'stone',	 'suitland',	 'takoma',	 'triangle',	 'upper',	 'vienna',	 'waldorf',	 'washingtondc',	 'wheaton',	 'woodbridge',	 'bethesda',	 'chase',	 'church',	 'college',	 'dc',
 'falls',	 'ferry',	 'heights',	 'hill',	 'marlboro',	 'park',	 'potomac',	 'ridge',	 'spring',	 'village',	 'washington']

In [ ]:
feature_cols = [
#'city',
#'country',
'latitude',
    #'location_data_accuracy',
'longitude',

#'state',
#'availability',
    #'average_image_size',
'bathroom',
'bedroom',
    

    #'cat',
#'date_available',
#'description',
    #'dog',
#'housing_type',
'image_number',
#'laundry',
#'parking',
    #'smoking',
'square_footage',    

#'url',
'grocery_list',
'gym_list',
#'movie_theatre_list',
#'train_station_list',
#'airport_list',
#'subway_station_list',
#'Barnes_and_Nobles_list',
#'Deloitte_list',
#'Starbucks_list',
    
#'time_of_day_of_posting',
#'date_of_posting',
    #'No_Housing_Data',
    #'apartment',
    #'condo',
    #'cottage/cabin',
    #'duplex',
    #'flat',
    #'house',
    #'in-law',
    #'townhouse',
    #'No_Laundry_Data',
    #'laundry in bldg',
    #'laundry on site',
    #'w/d hookups',
    #'w/d in unit',
    #'No_Parking_Data'
    
'arlington',
'dc',
    
'attached garage',
    #'carport',
    #'detached garage',
'off-street parking',
'street parking', 
    

]

set(data.park)

## Linear Regression

In [10]:
### SCIKIT-LEARN ###

# create X and y
X = data[feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

# instantiate and fit
lm2 = LinearRegression()
lm2.fit(X_train, y_train)

y_pred = lm2.predict(X_test) # Predict
# print the intercept
#print lm2.intercept_

# pair the feature names with the coefficients
#print zip(feature_cols, lm2.coef_)

# Access accuracy
y_pred = [int(y) for y in y_pred]
y_test = [int(y) for y in y_test]

print metrics.accuracy_score(y_test, y_pred)

0.000973709834469


## Logistic Regression

In [8]:
# create X and y

#feature_cols = []

X = data[feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

# Fit model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test) # Predict

# Access accuracy
print metrics.accuracy_score(y_test, y_pred)

0.246348588121


In [89]:
# Create a Confusion Matrix

con_mat = metrics.confusion_matrix(y_test, y_pred)
print con_mat

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [ ]:
##Random Forest

In [ ]:
# import class, instantiate estimator, fit with all data
rfclf = RandomForestClassifier(n_estimators=200, max_features=7, oob_score=True, random_state=1)
rfclf.fit(X_train, y_train)

# compute the out-of-bag classification accuracy
rfclf.oob_score_

print len(feature_cols)
print len(rfclf.feature_importances_)

In [ ]:
# compute the feature importances
Feature_Importance = pd.DataFrame({'feature':feature_cols, 'importance':rfclf.feature_importances_})
Feature_Importance.sort('importance', ascending=False)

In [ ]:
#Create Train Test split for text data (listing description)

naive_bayes_feature_cols = 'description'

# create X and y
X = data[naive_bayes_feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [ ]:
# COUNTVECTORIZER: 'convert text into a matrix of token counts'
# instantiate the vectorizer
vect = CountVectorizer()

# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train)
train_dtm

# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test)
test_dtm

# store feature names and examine them
train_features = vect.get_feature_names()
len(train_features)
train_features[:50]
train_features[-50:]

# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

# examine the vocabulary and document-term matrix together
pd.DataFrame(train_dtm.toarray(), columns=vect.get_feature_names()).head()

In [ ]:
nb = MultinomialNB()
nb.fit(train_dtm, y_train)

# make predictions on test data using test_dtm
y_pred = nb.predict(test_dtm)
y_pred

# compare predictions to true labels
print metrics.accuracy_score(y_test, y_pred)
#print metrics.confusion_matrix(y_test, y_pred)